In [8]:
## 윈도우 창 사이즈 조절 함수
def reset_window_size(window_name, img):
    ...
    
# 이미지 초기화 함수
def init_img_and_mask():
    ...
    
def resize_img(ratio, param):
    ... 
    
# roi를 선택해 반환하는 함수
def roi_copy(window_name, img_from):
    ... 
    
# 역투영 기능을 수행하는 함수
def back_project(window_name, img_copy):
    ...
    
# 받아온 마스크와 겹치는 이미지 미리보기 출력
def preview(img_copy, img_mask):
    ...      
    
# 읽어온 마스크를 img_binary_mask에 더하거나 빼는 함수
def save_to_mask(img_from, flag):
    ...
    
# 이미지를 흑백으로 변환해 이진화 시키는 기능을 수행하는 함수
def binary_threshold(window_name, img_copy):
    ...       
    
def hsv_masking(window_name, img_copy):
    ...       
    
#  마스킹 창에서 마우스 클릭시 호출되는 함수
def mouse_callback_masking(event, x, y, flags, param): 
    ...                
# 쓰레시홀드 트랙바 생성을 위한 콜백함수
def for_threshold_trackbar(x):
    ...
# hsv 값에 대한 트랙바 생성을 위한 콜백함수
def for_hsv_trackbar(x):
    ...

######################## main 부분 ########################
import cv2
import numpy as np

# 해당 경로에 이미지가 존재할 때 까지 반복
...

# 프로세스 수행 반복, 키보드 입력으로 다음 기능을 선택
while True:
    # 키보드 입력으로 작업 선택 [esc,+,-,1,2,3,i,p,q,r,s]
    ...
    # q - 닫기
    ... 
    # + - 그림 확대
    ...
    # - - 그림 축소
    ...    
    # i - Current IMG 초기 이미지로 초기화
    ...    
    # p - 저장될 이미지 미리보기
    ...        
    # s - 저장
    ...               
    # r - roi 영역 선택
    ...        
    # 1번 - 역투영 방식 선택했을 때
    ...        
    # 2번 - 흑백 사진에 대한 쓰레시 홀드를 통한 이분화작업 수행
    ...        
    # 3번 - hsv 마스킹 방식 눌렀을 때
    ...        
                
print("종료합니다.")
# 이미지 경로 예시 : img/t3.jpg'

IndentationError: expected an indented block (<ipython-input-8-852510b041eb>, line 9)

In [6]:
def test():
    global value
    value=0
    print(value)
    
test()
print(value)

0
0


In [ ]:
import cv2
import numpy as np

img = np.full((900,900,4), 255, dtype=np.uint8)
img[:, :, 3] = 0
cv2.imshow('test', img)
cv2.imwrite('img/blank_900.png', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
######################## 함수 define 부분 ########################

def mouse_callback_cody(event, x, y, flags, param):
    global img_clothes, img_before, img_add, not_alpha_area, w, h
    img_roi = img_add.copy()
    y -= h//2
    x -= w//2
    min_x = 0
    min_y = 0
    max_y = img_clothes.shape[0]
    max_x = img_clothes.shape[1]
    
    # 우클릭 눌렸을 떄
    if event == cv2.EVENT_RBUTTONDOWN:
        img_before = img_clothes.copy() # 다시 되돌아가기 위해 필요한 임시 저장 변수
        
        # 오른쪽 위로 넘어갔을 때
        if x+w> max_x and y < 0:
            img_roi = img_roi[(-y):, :-(x+w-max_x)]
            max_x -= x
            min_y += -y
            img_clothes[:y+h,x:x+w] = cv2.bitwise_and(img_clothes[:y+h,x:x+w], img_clothes[:y+h,x:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,x:x+w] = cv2.add(img_clothes[:y+h,x:x+w], img_roi)
        
        # 오른쪽 아래로 넘어갔을 때
        elif y+h > max_y and x+w > max_x:
            img_roi = img_roi[:-(y+h-max_y), :-(x+w-max_x)]
            max_y -= y
            max_x -= x
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w], 
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
        # 왼쪽 위로 넘어갔을 때
        elif y < 0 and x < 0:
            img_roi = img_roi[(-y):, (-x):]
            min_y += -y
            min_x += -x
            img_clothes[:y+h,:x+w] = cv2.bitwise_and(img_clothes[:y+h,:x+w], img_clothes[:y+h,:x+w], 
                                                     mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,:x+w] = cv2.add(img_clothes[:y+h,:x+w], img_roi) 
            
        # 왼쪽 아래로 넘어갔을 때
        elif x < 0 and y+h > max_y:
            img_roi = img_roi[:-(y+h-max_y), (-x):]
            max_y -= y
            min_x += -x
            img_clothes[y:y+h,:x+w] = cv2.bitwise_and(img_clothes[y:y+h,:x+w], img_clothes[y:y+h,:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,:x+w] = cv2.add(img_clothes[y:y+h,:x+w], img_roi)
        
        # 위로 넘어갔을 때
        elif y < 0:
            img_roi = img_roi[(-y):, :]
            min_y += -y
            img_clothes[:y+h,x:x+w] = cv2.bitwise_and(img_clothes[:y+h,x:x+w], img_clothes[:y+h,x:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,x:x+w] = cv2.add(img_clothes[:y+h,x:x+w], img_roi)
          
        # 아래로 넘어갔을 때
        elif y+h > max_y:
            img_roi = img_roi[:-(y+h-max_y), :]
            max_y -= y
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w],
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
            
        # 왼쪽으로 넘어갔을 때
        elif x < 0:
            img_roi = img_roi[:, (-x):]
            min_x += -x
            img_clothes[y:y+h,:x+w] = cv2.bitwise_and(img_clothes[y:y+h,:x+w], img_clothes[y:y+h,:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,:x+w] = cv2.add(img_clothes[y:y+h,:x+w], img_roi)
        
        # 오른쪽으로 넘어갔을 때
        elif x+w> max_x:
            img_roi = img_roi[:, :-(x+w-max_x)]
            max_x -= x
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w],
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
        # 정상적으로 내부에 들어왔을 때
        else:    
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w], 
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
    # 우클릭 뗏을 때
    elif event == cv2.EVENT_RBUTTONUP:
        img_clothes = img_before.copy()
        
    # 좌클릭 했을 때
    elif event == cv2.EVENT_LBUTTONDOWN:
        # 오른쪽 위로 넘어갔을 때
        if x+w> max_x and y < 0:
            img_roi = img_roi[(-y):, :-(x+w-max_x)]
            max_x -= x
            min_y += -y
            img_clothes[:y+h,x:x+w] = cv2.bitwise_and(img_clothes[:y+h,x:x+w], img_clothes[:y+h,x:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,x:x+w] = cv2.add(img_clothes[:y+h,x:x+w], img_roi)
        
        # 오른쪽 아래로 넘어갔을 때
        elif y+h > max_y and x+w > max_x:
            img_roi = img_roi[:-(y+h-max_y), :-(x+w-max_x)]
            max_y -= y
            max_x -= x
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w], 
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
        # 왼쪽 위로 넘어갔을 때
        elif y < 0 and x < 0:
            img_roi = img_roi[(-y):, (-x):]
            min_y += -y
            min_x += -x
            img_clothes[:y+h,:x+w] = cv2.bitwise_and(img_clothes[:y+h,:x+w], img_clothes[:y+h,:x+w], 
                                                     mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,:x+w] = cv2.add(img_clothes[:y+h,:x+w], img_roi) 
            
        # 왼쪽 아래로 넘어갔을 때
        elif x < 0 and y+h > max_y:
            img_roi = img_roi[:-(y+h-max_y), (-x):]
            max_y -= y
            min_x += -x
            img_clothes[y:y+h,:x+w] = cv2.bitwise_and(img_clothes[y:y+h,:x+w], img_clothes[y:y+h,:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,:x+w] = cv2.add(img_clothes[y:y+h,:x+w], img_roi)
        
        # 위로 넘어갔을 때
        elif y < 0:
            img_roi = img_roi[(-y):, :]
            min_y += -y
            img_clothes[:y+h,x:x+w] = cv2.bitwise_and(img_clothes[:y+h,x:x+w], img_clothes[:y+h,x:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,x:x+w] = cv2.add(img_clothes[:y+h,x:x+w], img_roi)
          
        # 아래로 넘어갔을 때
        elif y+h > max_y:
            img_roi = img_roi[:-(y+h-max_y), :]
            max_y -= y
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w],
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
            
        # 왼쪽으로 넘어갔을 때
        elif x < 0:
            img_roi = img_roi[:, (-x):]
            min_x += -x
            img_clothes[y:y+h,:x+w] = cv2.bitwise_and(img_clothes[y:y+h,:x+w], img_clothes[y:y+h,:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,:x+w] = cv2.add(img_clothes[y:y+h,:x+w], img_roi)
        
        # 오른쪽으로 넘어갔을 때
        elif x+w> max_x:
            img_roi = img_roi[:, :-(x+w-max_x)]
            max_x -= x
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w],
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
        # 정상적으로 내부에 들어왔을 때
        else:    
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w], 
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)


######################## main 부분 ########################
import cv2
import numpy as np

img = cv2.imread('img/blank_900.png', cv2.IMREAD_UNCHANGED) # 알파채널까지 그대로 읽어오기
img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
img_clothes = img.copy() 
img_before = img_clothes.copy() # 미리보기 기능 수행을 위한 임시 변수
key = 97 # 초기값 'a'의 ascii code

while True:    
    # q - 종료
    if key == 113:
        cv2.destroyAllWindows()
        break
    # s - 저장
    elif key == ord('s'):
        cv2.imwrite('img/clothes_cody.png', img_clothes)
    # i - 이미지 초기화
    elif key == ord('i'):
        img_clothes = img.copy()
    # a - 새 이미지 추가
    elif key == ord('a'):
        while True:
            img_file = input("이미지 경로 :") #'img/t4.png' #
            img_add = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
            if img_add is None:
                print("경로에 이미지가 존재하지 않습니다.")
                continue
                
            cv2.imshow('Cody', img_clothes)
            cv2.setMouseCallback('Cody', mouse_callback_cody) # 마우스 콜백 함수 등록
            h, w = img_add.shape[:2] # 가로와 세로 길이
            _, alpha_area = cv2.threshold(img_add[:,:,3], 1, 255, cv2.THRESH_BINARY) # 알파채널에 대한 이진화
            not_alpha_area = cv2.bitwise_not(alpha_area) # 배경부분 선택
            img_add = cv2.bitwise_and(img_add, img_add, mask=alpha_area) # 더할 이미지 선택
            break
            
    cv2.imshow('Cody', img_clothes)
    key = cv2.waitKey(1)
cv2.destroyAllWindows()

# 이미지 경로 예시 : img/p3.png, img/t3.png

새 이미지 추가
이미지 경로 :img/p3.png
하
중앙
중앙
중앙
중앙
우하
우상
좌상
좌하
하
상
우
중앙
중앙
중앙
중앙
하
이미지 초기화
중앙
중앙
중앙
새 이미지 추가
이미지 경로 :img/t3.png
중앙
중앙
중앙
중앙
이미지 저장
중앙
종료
